In [1]:
 pip install SQLAlchemy==1.4.46 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10


In [2]:
pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=854ca00a961945c92b847e37c3e888e7e268f22834a2e7357a5b1e62fa76af6d
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [6]:
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from google.colab import drive

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


## Business Case

In [8]:
rental_data = pd.read_excel('/content/drive/MyDrive/Sql_Business_Case/Sql_Business_Case.xlsx')
rental_data

,ID_RIDE,ID_USER,ID_VEHICLE,RIDE_CITY,RIDE_START_DATETIME,RIDE_END_DATETIME,DRIVEN_DISTANCE_KM
0,-8.737.736.313.567.877.182,6.719.092.602.775.602.517,NaN,Hamburg,2018-09-04 23:54:21.553,2018-09-09 22:20:31.953,"783,125"
1,232.875.352.962.553.906,-5.616.694.520.543.515.123,8.643.784.747.205.842.033,Berlin,2018-09-04 23:51:36.060,2018-09-04 23:52:51.390,0
2,-1.975.628.777.468.322.805,2.223.274.508.386.310.967,-2.776.475.389.162.094.752,Berlin,2018-09-04 23:47:01.633,2018-09-04 23:57:30.440,"7,35"
3,-4.660.060.998.223.104.203,-9.217.133.333.851.900.657,-8.535.074.439.992.352.969,Berlin,2018-09-04 23:46:14.397,2018-09-05 00:11:01.877,"19,425"
4,8.023.310.024.128.545.133,-7.346.307.447.472.267.304,-71.875.240.866.433.689,Berlin,2018-09-04 23:28:21.057,2018-09-04 23:49:14.930,"12,425"
...,...,...,...,...,...,...,...
13979,-5.731.302.821.613.716.580,3.471.864.425.953.845.592,-303.268.269.945.295.252,Berlin,2018-08-01 03:19:06.277,2018-08-01 04:29:23.707,"52,5"
13980,5.793.441.748.402.952.243,-4.930.908.301.296.568.524,3.014.847.247.466.281.620,Berlin,2018-08-01 03:07:42.710,2018-08-01 03:19:30.080,"8,4"
13981,762.041.063.260.980.198,-2.100.271.122.499.853.667,-4.535.449.754.272.503.040,Berlin,2018-08-01 02:12:13.813,2018-08-01 02:32:00.377,"12,25"
13982,3.688.212.783.127.062.184,2.827.966.154.436.069.035,-2.892.561.419.585.835.562,Hamburg,2018-08-01 01:34:51.440,2018-08-01 01:36:16.920,175


In [9]:
rental_data.dtypes

ID_RIDE                        object
ID_USER                        object
ID_VEHICLE                     object
RIDE_CITY                      object
RIDE_START_DATETIME    datetime64[ns]
RIDE_END_DATETIME      datetime64[ns]
DRIVEN_DISTANCE_KM             object
dtype: object

In [10]:
rental_data = ps.sqldf(""" 
                          SELECT
                              ID_RIDE,
                              ID_USER,
                              ID_VEHICLE,
                              RIDE_CITY,
                              RIDE_START_DATETIME,
                              RIDE_END_DATETIME,
                              CAST(DRIVEN_DISTANCE_KM AS FLOAT) AS DRIVEN_DISTANCE_KM
                            FROM (
                            SELECT
                                ID_RIDE,
                                ID_USER,
                                ID_VEHICLE,
                                RIDE_CITY,
                                RIDE_START_DATETIME,
                                RIDE_END_DATETIME,
                                REPLACE(DRIVEN_DISTANCE_KM, ',' ,'.') AS DRIVEN_DISTANCE_KM
                            FROM rental_data
                            ) as tb1
                            ORDER BY RIDE_START_DATETIME
                                """)
rental_data

,ID_RIDE,ID_USER,ID_VEHICLE,RIDE_CITY,RIDE_START_DATETIME,RIDE_END_DATETIME,DRIVEN_DISTANCE_KM
0,8.333.288.488.516.743.004,7.930.224.137.303.910.526,-7.691.483.099.551.667.362,Berlin,2018-08-01 00:05:06.833000,2018-08-01 00:14:52.300000,6.825
1,3.688.212.783.127.062.184,2.827.966.154.436.069.035,-2.892.561.419.585.835.562,Hamburg,2018-08-01 01:34:51.440000,2018-08-01 01:36:16.920000,175.000
2,762.041.063.260.980.198,-2.100.271.122.499.853.667,-4.535.449.754.272.503.040,Berlin,2018-08-01 02:12:13.813000,2018-08-01 02:32:00.377000,12.250
3,5.793.441.748.402.952.243,-4.930.908.301.296.568.524,3.014.847.247.466.281.620,Berlin,2018-08-01 03:07:42.710000,2018-08-01 03:19:30.080000,8.400
4,-5.731.302.821.613.716.580,3.471.864.425.953.845.592,-303.268.269.945.295.252,Berlin,2018-08-01 03:19:06.277000,2018-08-01 04:29:23.707000,52.500
...,...,...,...,...,...,...,...
13979,8.023.310.024.128.545.133,-7.346.307.447.472.267.304,-71.875.240.866.433.689,Berlin,2018-09-04 23:28:21.057000,2018-09-04 23:49:14.930000,12.425
13980,-4.660.060.998.223.104.203,-9.217.133.333.851.900.657,-8.535.074.439.992.352.969,Berlin,2018-09-04 23:46:14.397000,2018-09-05 00:11:01.877000,19.425
13981,-1.975.628.777.468.322.805,2.223.274.508.386.310.967,-2.776.475.389.162.094.752,Berlin,2018-09-04 23:47:01.633000,2018-09-04 23:57:30.440000,7.350
13982,232.875.352.962.553.906,-5.616.694.520.543.515.123,8.643.784.747.205.842.033,Berlin,2018-09-04 23:51:36.060000,2018-09-04 23:52:51.390000,0.000


## BUSINESS CASE: SQL REVIEW (1/3)

 **How many rides and customers were there in each city on the Friday of the first week in the given dataset?**

In [11]:
## Number of rides and customer on Friday of the first week
no_customers_and_rides = ps.sqldf("""
                                    SELECT
                                        ride_city AS city,
                                        COUNT(ID_USER) AS total_users,
                                        COUNT(ID_VEHICLE) AS total_vehicles
                                    FROM rental_data
                                    WHERE DATE(RIDE_START_DATETIME) = '2018-08-03'
                                    GROUP BY RIDE_CITY
                                    """)
no_customers_and_rides

,city,total_users,total_vehicles
0,Berlin,358,355
1,Hamburg,65,62


In [12]:
"""
I wasn't sure if you meant here Friday of first week (which means we just add the date) or the friday of 
first week for each month. Hence, I decided to do find friday of first week for each month
"""

"""
I first count the total users and vehicles by date and city. Then for each date, I check 
what day it is for example Monday or Tuesday. Then I find the minimum date when Friday came for each month.
There are multiple ways of doing it but sql in pandas doesn't have a lot of functionalities hence I had to use
case solution to check the date
"""


no_customers_and_rides_first_week_each_month = ps.sqldf("""
                                                WITH rides_and_customer_per_day AS (
                                                
                                                    SELECT
                                                        DATE(RIDE_START_DATETIME) AS date,
                                                        ride_city AS city,
                                                        COUNT(ID_USER) AS total_users,
                                                        COUNT(ID_VEHICLE) AS total_vehicles
                                                    FROM rental_data
                                                    GROUP BY  date, city
                                                    ORDER BY date, city
                                                
                                                ),
                                                
                                                days_by_date AS (
                                                    SELECT 
                                                    CASE CAST(strftime('%w', date) AS INTEGER)
                                                        WHEN 0 THEN 'Sunday'
                                                        WHEN 1 THEN 'Monday'
                                                        WHEN 2 THEN 'Tuesday'
                                                        WHEN 3 THEN 'Wednesday'
                                                        WHEN 4 THEN 'Thursday'
                                                        WHEN 5 THEN 'Friday'
                                                        ELSE 'Saturday'
                                                    END AS day,
                                                    date
                                                    FROM rides_and_customer_per_day
                                                )
                                                
                                                SELECT
                                                    city,
                                                    MIN(date) AS date,
                                                    day,
                                                    total_users,
                                                    total_vehicles
                                                FROM (
                                                    SELECT 
                                                        rc.date AS date,
                                                        dd.day AS day,
                                                        rc.city AS city,
                                                        rc.total_users AS total_users,
                                                        rc.total_vehicles AS total_vehicles
                                                    FROM rides_and_customer_per_day AS rc
                                                    INNER JOIN days_by_date AS dd
                                                    ON rc.date = dd.date
                                                    ) AS tb1
                                                WHERE day = 'Friday'
                                                GROUP BY strftime('%m', date), city

                                                """)

no_customers_and_rides_first_week_each_month

,city,date,day,total_users,total_vehicles
0,Berlin,2018-08-03,Friday,358,355
1,Hamburg,2018-08-03,Friday,65,62


## BUSINESS CASE: SQL REVIEW (2/3)

**How many new activations (new first-time users) were there in each month in the given dataset?**

In [ ]:
first_time_users  = ps.sqldf("""
                                WITH total_ride_by_customer AS (
                                SELECT
                                    month,
                                    COUNT(ID_USER) AS total_new_users
                                FROM 
                                (
                                    SELECT
                                        ID_USER,
                                        RIDE_START_DATETIME,
                                        strftime('%m', RIDE_START_DATETIME) AS month,
                                        DENSE_RANK() OVER (PARTITION BY ID_USER ORDER BY RIDE_START_DATETIME) AS order_number 
                                    FROM rental_data
                                    WHERE ID_USER != 'None'
                                ) as tb1
                                WHERE order_number = 1
                                GROUP BY month
                            )
                                SELECT * FROM total_ride_by_customer
                                """)
first_time_users

,month,total_new_users
0,08,8263
1,09,887


## BUSINESS CASE: SQL REVIEW (3/3)

**Use the data in the case study to create the following output using SQL:**
- **user_id**
- **ride_id**
- **ride_start**
- **Time a car is not used between two rides**
- Time between two rides by customers

In [13]:
last_case = ps.sqldf("""
                        WITH car_not_used_between_ride AS (
                            SELECT
                                vehicle_id,
                                user_id,
                                ride_start_date,
                                ride_end_date,
                                ride_end_previous_period,
                                strftime('%s',ride_start_date) - strftime('%s',ride_end_previous_period) AS time_car_not_used
                            FROM (
                                SELECT
                                    ID_VEHICLE AS vehicle_id,
                                    ID_USER AS user_id,
                                    RIDE_START_DATETIME AS ride_start_date,
                                    RIDE_END_DATETIME AS ride_end_date,
                                    LAG(RIDE_END_DATETIME) OVER (PARTITION BY ID_VEHICLE ORDER BY RIDE_START_DATETIME) AS ride_end_previous_period 
                                FROM rental_data
                                ORDER BY RIDE_START_DATETIME
                            ) as tb1
                        ),
                        
                        
                        time_between_two_ride_by_customer AS (
                        SELECT
                            user_id,
                            vehicle_id,
                            ride_start_date,
                            ride_end_date,
                            ride_end_previous_period,
                            strftime('%s',ride_start_date) - strftime('%s',ride_end_previous_period) AS time_between_rides_by_customer
                        FROM (
                            SELECT
                                ID_USER AS user_id,
                                ID_VEHICLE AS vehicle_id,
                                RIDE_START_DATETIME AS ride_start_date,
                                RIDE_END_DATETIME AS ride_end_date,
                                LAG(RIDE_END_DATETIME) OVER (PARTITION BY ID_USER ORDER BY RIDE_START_DATETIME) AS ride_end_previous_period 
                            FROM rental_data
                            ORDER BY RIDE_START_DATETIME
                            ) AS tb1
                        
                        )
                        
                        SELECT
                            t1.user_id,
                            t1.vehicle_id,
                            t1.ride_start_date,
                            COALESCE(t1.time_car_not_used, 0) AS time_car_not_used,
                            COALESCE(t2.time_between_rides_by_customer, 0) AS time_between_rides_by_customer
                        FROM car_not_used_between_ride AS t1
                        INNER JOIN time_between_two_ride_by_customer AS t2 ON t1.user_id = t2.user_id AND t1.vehicle_id = t2.vehicle_id AND t1.ride_start_date = t2.ride_start_date
                        GROUP BY t1.user_id, t1.vehicle_id, t1.ride_start_date
                        ORDER BY  t1.ride_start_date
                            
                        
                    """)
last_case

,user_id,vehicle_id,ride_start_date,time_car_not_used,time_between_rides_by_customer
0,7.930.224.137.303.910.526,-7.691.483.099.551.667.362,2018-08-01 00:05:06.833000,0,0
1,2.827.966.154.436.069.035,-2.892.561.419.585.835.562,2018-08-01 01:34:51.440000,0,0
2,-2.100.271.122.499.853.667,-4.535.449.754.272.503.040,2018-08-01 02:12:13.813000,0,0
3,-4.930.908.301.296.568.524,3.014.847.247.466.281.620,2018-08-01 03:07:42.710000,0,0
4,3.471.864.425.953.845.592,-303.268.269.945.295.252,2018-08-01 03:19:06.277000,0,0
...,...,...,...,...,...
13734,3.933.796.054.101.688.663,2.972.016.530.587.358.975,2018-09-04 23:16:47.523000,7557,0
13735,-7.346.307.447.472.267.304,-71.875.240.866.433.689,2018-09-04 23:28:21.057000,44501,896636
13736,-9.217.133.333.851.900.657,-8.535.074.439.992.352.969,2018-09-04 23:46:14.397000,31971,0
13737,2.223.274.508.386.310.967,-2.776.475.389.162.094.752,2018-09-04 23:47:01.633000,2674,2674
